In [1]:
# -*- coding: utf-8 -*-
"""
@Author: Guan-Fu Liu
Created on Jan. 14, 2024
Last modified on May 5, 2024

Convert the stellar yield file from Sukhbold et al. (2016) to the format more convenient for 
further IMF determination.

I decide not to convert the mass to number

Note that the yields from Sukhbold et al. (2016) are total yields.
We can get the net yields by minusing the progenitor model.

You can download the yields table and progenitor models from
https://wwwmpa.mpa-garching.mpg.de/ccsnarchive/data/SEWBJ_2015/index.html
"""
import numpy as np
import pandas as pd
import pyatomdb
import re
import os
from functools import reduce

In [2]:
def get_S16_single_yields(file):
    """
    Get the yileds from S16.
    
    Parameters
    ----------
    file : str
        The file name of the yield table.
    
    Returns
    -------
    yields : dataframe
        The dataframe of the yields.
        Its indice are "H, He, Li ...".
        Its column is "yield (ejecta+wind)", whose name is the initial mass.
    """
    # Get the number between 's' and '.yield_table', which is the initial mass.
    pattern = re.compile(r's(\d+(?:\.\d+)?).yield_table')
    match = pattern.search(file)
    m_ini = match.group(1)
    if 'implosions_W18' in file:
        y = np.loadtxt(file, skiprows=1, dtype=[('isotope', '<U10'), ('wind', 'f8')])

        y = pd.DataFrame(y['wind'], index=y['isotope'], columns=[m_ini])
    else:
        y = np.loadtxt(file, skiprows=1, dtype=[('isotope', '<U10'), ('ejecta', 'f8'), ('wind', 'f8')])[:284]
        #  [:284] is to remove the unstable isotopes
        # y = pd.DataFrame(y['ejecta']+y['wind'], index=y['isotope'], columns=[m_ini])
        y = pd.DataFrame(y['ejecta']+y['wind'], index=y['isotope'], columns=[m_ini])
        # y = pd.DataFrame(y['ejecta'], index=y['isotope'], columns=[m_ini])
    y.index = y.index.str.capitalize()
    y['element'] = y.index.str.extract(r'([A-Za-z]+)').to_numpy().ravel()  # The element symbol
    y['A'] = y.index.str.extract(r'(\d+)').to_numpy().ravel()  # The mass number
    # y[m_ini] = y[m_ini]/y['A'].astype(float)  # The yield in number
    y = y.groupby('element')[m_ini].sum().reset_index()
    y['Z'] = y['element'].apply(pyatomdb.atomic.elsymb_to_Z)
    y.sort_values(by='Z', inplace=True)
    y.drop('Z', axis=1, inplace=True)
    y.rename(columns={'element':'M'}, inplace=True)
    y.set_index('M', inplace=True)
    return y


def merge_two_dfs(left, right):
    """
    Define a function to merge two DataFrames
    """
    return pd.merge(left, right, how='outer')

# From Tuguldur Sukhbold:

This being said, I can address your specific question - when considering different engines we have segregated the stars into two categories:

- $M_{\mathrm{ZAMS}}\leq 12 M_{\mathrm{\odot}}$: we considered them as "Crab like"
- $M_{\mathrm{ZAMS}}> 12 M_{\mathrm{\odot}}$: we considered them as "87A like"

for stars with $\leq 12 M_{\mathrm{\odot}}$ we only applied the Z9.6 engine, and for heavier stars we applied 4 different engines (W18, N20, S19.8 etc...) to get the explosion outcomes. And when computing integrated yields we combine Z9.6 results for lighter stars with W18 or N20 engine results of heavier stars, e.g.:

- combination 1: Z9.6 (for $\leq 12 M_{\mathrm{\odot}}$) + W18 (for $> 12 M_{\mathrm{\odot}}$)
- combination 2: Z9.6 (for $\leq 12 M_{\mathrm{\odot}}$) + N20 (for $> 12 M_{\mathrm{\odot}}$)

In [3]:
S16mods = ["W18", "N20"]  # The yields in Z9.6 have been copied and pasted to the W18 and N20 folder.
for S16mod in S16mods:
    files = os.listdir('./Original/S16/%s/' % S16mod)
    files = [item for item in files if item.endswith('.yield_table')]
    tables = [ ]
    for file in files:
        tables.append(get_S16_single_yields('./Original/S16/%s/%s' % (S16mod, file)))
    merged_df = pd.concat(tables, axis=1)
    cols = merged_df.columns
    cols1 = np.array([float(item) for item in cols])
    cols = cols[cols1.argsort()]
    # cols = cols.insert(0, 'element')
    merged_df = merged_df[cols]
    merged_df.to_csv('./SNcc/S16%s-0.0200-SNcc-total.csv' % S16mod, index=True, float_format='%.2e')
    # The float format of original yield table is '%.2e'.

In [4]:
merged_df.sum(axis=0)

9.0       7.636226
9.25      7.839459
9.5       8.079141
9.75      8.282657
10.0      8.511839
           ...    
29.6     27.766664
60       58.226861
80       78.070826
100      97.997928
120     117.925307
Length: 118, dtype: float64

In [5]:
merged_df

,9.0,9.25,9.5,9.75,10.0,10.25,10.5,10.75,11.0,11.25,...,27.3,27.4,29.0,29.1,29.2,29.6,60,80,100,120
M,,,,,,,,,,,,,,,,,,,,,
H,4.718031e+00,4.830031e+00,4.954031e+00,5.063031e+00,5.167031e+00,5.265031e+00,5.342030e+00,5.434030e+00,5.534030e+00,5.688030e+00,...,1.084000e+01,1.089000e+01,1.130000e+01,1.138000e+01,1.140000e+01,1.152000e+01,2.020000e+01,2.550000e+01,3.060000e+01,3.590000e+01
He,2.743358e+00,2.818560e+00,2.904563e+00,2.988465e+00,3.064568e+00,3.164868e+00,3.270969e+00,3.372571e+00,3.463774e+00,3.538281e+00,...,8.250615e+00,8.220618e+00,8.830634e+00,8.700643e+00,8.780642e+00,8.870650e+00,2.232113e+01,3.666128e+01,5.149135e+01,6.418136e+01
Li,1.187548e-07,1.648170e-07,2.228926e-07,2.539391e-07,2.550070e-07,2.730423e-07,2.850119e-07,3.030479e-07,3.490918e-07,3.322324e-07,...,2.460893e-07,3.070957e-07,5.562514e-07,5.562626e-07,6.092736e-07,6.033258e-07,9.438250e-08,1.233575e-07,9.957150e-08,1.059424e-07
Be,2.320000e-10,2.431000e-10,2.673000e-10,2.583000e-10,2.535000e-10,2.531000e-10,2.420000e-10,2.468000e-10,2.586000e-10,2.521000e-10,...,1.311000e-10,1.281000e-10,1.338000e-10,1.355000e-10,1.370000e-10,1.382000e-10,2.476000e-10,2.947000e-10,3.566000e-10,4.714000e-10
B,1.600300e-07,2.550707e-07,3.993450e-07,4.313620e-07,4.297390e-07,4.257740e-07,4.712120e-07,5.102540e-07,6.563450e-07,5.983060e-07,...,2.160460e-06,2.410420e-06,1.699910e-06,1.719780e-06,1.830050e-06,1.830050e-06,2.137900e-06,1.592160e-06,1.527600e-06,1.623960e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Au,8.628000e-09,8.858000e-09,9.111000e-09,9.295000e-09,9.548000e-09,9.806000e-09,1.009700e-08,1.033100e-08,1.055300e-08,1.080900e-08,...,2.330000e-08,2.319000e-08,2.448000e-08,2.435000e-08,2.444000e-08,2.465000e-08,6.233000e-08,9.175000e-08,1.178000e-07,1.428000e-07
Hg,1.930762e-08,1.978548e-08,2.065046e-08,2.130750e-08,2.210627e-08,2.318357e-08,2.408771e-08,2.518562e-08,2.537273e-08,2.653410e-08,...,7.473420e-08,7.443470e-08,8.184020e-08,8.175350e-08,8.128270e-08,8.251600e-08,2.693800e-07,3.309590e-07,3.971170e-07,4.753950e-07
Tl,8.541600e-09,8.770300e-09,9.123000e-09,9.297000e-09,9.590000e-09,9.928000e-09,1.038850e-08,1.069300e-08,1.112500e-08,1.139900e-08,...,3.934000e-08,3.998000e-08,4.006000e-08,3.968000e-08,4.048000e-08,4.033000e-08,7.833000e-08,9.780000e-08,1.175500e-07,1.412400e-07
